In [ ]:
import cv2
import threading
import time
import os
import dlib

# Thread-safe variable
stop_thread = False
lock = threading.Lock()

def set_stop_thread():
    global stop_thread
    time.sleep(10)  # Wait for 10 seconds
    with lock:
        stop_thread = True

def face_registration(username: str):
    # Start the thread that sets the stop_thread variable after 10 seconds
    stop_thread_thread = threading.Thread(target=set_stop_thread)
    stop_thread_thread.start()

    # Ensure the directory exists
    output_directory = f'../UserFaces/{username}/'
    os.makedirs(output_directory, exist_ok=True)

    video_path = os.path.join(output_directory, 'video.avi')

    capture = cv2.VideoCapture(0)

    fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
    videoWriter = cv2.VideoWriter(video_path, fourcc, 30.0, (640, 480), True)

    while True:
        ret, frame = capture.read()

        if ret:
            cv2.imshow('Webcam', frame)
            videoWriter.write(frame)

        with lock:
            if stop_thread:
                break

        if cv2.waitKey(1) == 27:
            break

    capture.release()
    videoWriter.release()

    cv2.destroyAllWindows()
    stop_thread_thread.join()  # Wait for the stop_thread thread to finish

    detect_faces(username)
    #os.remove(f'../UserFaces/{username}/video.avi')


In [ ]:
def process_frame(frame):
    face_detector = dlib.get_frontal_face_detector() # Model used to get detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the image to grayscale
    faces = face_detector(gray, 1) # Detect faces in the grayscale image

    if faces:
        return True

    return False

def detect_faces(username:str):

    output_directory = os.path.join('..', 'UserFaces', username)
    os.makedirs(output_directory, exist_ok=True)

    cap = cv2.VideoCapture(os.path.join(output_directory, 'video.avi'))

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    valid_frames = []
    frame_count = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            print("End of video.")
            break

        if(valid_frames != [] and frame_count - valid_frames[-1] < 10 and frame_count // 10 == valid_frames[-1] // 10):
            frame_count += 1
            continue

        if(process_frame(frame)): # Save the face in the folder
            valid_frames.append(frame_count)
            print(f'Valid frame detected: {frame_count}')
            cv2.imwrite(os.path.join(output_directory, f'face_{len(valid_frames)}.png'), frame)

        frame_count += 1 

    cap.release()

In [ ]:
def face_recognition():

    username = "Unknown"

    return username

In [ ]:
from Modules.game_communicator import GameCommunicator

game_communicator = GameCommunicator()
game_communicator.wait_for_connection()

while True:

    msg = game_communicator.receive_from_game()
    msg = msg.split(':')

    if msg[0] == "face_registration":
        username = msg[1]
        #face_registration(username)
        detect_faces(username)

        game_communicator.send_to_game("success")

    if msg == "face_recognition":

        username = face_recognition()
        game_communicator.send_to_game(username)